# Demo interattiva: Spettro di Riemann, $\lambda$ e $\Delta\Psi$

Questa demo mostra:
- Grafico interattivo di $\Delta\Psi$ in funzione di $\lambda$ (autovalore/spettro)
- Calcolo di $\Delta\Psi$ e confronto con valore atteso
- Esportazione dei dati in `.xlsx` come output coerente con i test automatici

Modifica i parametri e osserva i risultati in tempo reale!

In [1]:
# Import base
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider
import os
plt.style.use('seaborn-v0_8-darkgrid')

In [2]:
# Hamiltoniana semplice 2x2 e vettori
H = np.array([[2, 0], [0, 3]])
psi1 = np.array([1, 0])
psi2 = np.array([0, 1])
psi_mix = np.array([1, 1]) / np.sqrt(2)

def delta_psi(H, psi, lam):
    """Calcola il funzionale di coerenza ||Hψ - λψ||_2"""
    diff = np.dot(H, psi) - lam * psi
    return np.linalg.norm(diff)

def lambda_expected(H, psi):
    """Valore atteso di λ dato ψ"""
    num = np.vdot(psi, np.dot(H, psi))
    denom = np.vdot(psi, psi)
    return np.real(num / denom)

## 1. Grafico interattivo: $\Delta\Psi$ in funzione di $\lambda$

In [3]:
def plot_delta_psi(psi_choice):
    lambdas = np.linspace(1.5, 3.5, 300)
    if psi_choice == 'psi1':
        psi = psi1
        label = r"$\psi_1 = [1, 0]$ (autovalore 2)"
    elif psi_choice == 'psi2':
        psi = psi2
        label = r"$\psi_2 = [0, 1]$ (autovalore 3)"
    else:
        psi = psi_mix
        label = r"$\psi_{mix} = [1, 1]/\sqrt{2}$"
    dps = [delta_psi(H, psi, lam) for lam in lambdas]
    plt.figure(figsize=(7, 4))
    plt.plot(lambdas, dps, lw=2)
    plt.title(f"$\\Delta\\Psi$ per {label}", fontsize=15)
    plt.xlabel(r"$\lambda$")
    plt.ylabel(r"$\Delta\Psi$")
    plt.grid(True, alpha=0.4)
    plt.show()

interact(plot_delta_psi, psi_choice=['psi1', 'psi2', 'psi_mix']);

## 2. Calcolo diretto $\Delta\Psi$ e confronto con valore atteso

In [4]:
def demo_delta_psi(psi_type, lambdaval):
    if psi_type == 'psi1':
        psi = psi1
    elif psi_type == 'psi2':
        psi = psi2
    else:
        psi = psi_mix
    dpsi = delta_psi(H, psi, lambdaval)
    lam_exp = lambda_expected(H, psi)
    print(f"λ scelto: {lambdaval:.3f}")
    print(f"λ atteso (valore teorico): {lam_exp:.3f}")
    print(f"ΔΨ: {dpsi:.4e}")
    if np.isclose(dpsi, 0, atol=1e-8):
        print("\u2705 Coerenza perfetta: λ è autovalore di H per ψ.")
    else:
        print("\u26A0\ufe0f Coerenza non nulla: λ NON è autovalore per ψ.")

interact(demo_delta_psi, 
         psi_type=['psi1', 'psi2', 'psi_mix'],
         lambdaval=FloatSlider(value=2, min=1.5, max=3.5, step=0.01));

## 3. Esportazione risultati $\Delta\Psi$, $\lambda$ in `.xlsx`

In [5]:
# Simulazione export coerente .xlsx
lambdas = np.linspace(1.5, 3.5, 101)
results = []
for psi_label, psi in zip(['psi1', 'psi2', 'psi_mix'], [psi1, psi2, psi_mix]):
    dps = [delta_psi(H, psi, lam) for lam in lambdas]
    for lam, val in zip(lambdas, dps):
        results.append({'psi': psi_label, 'lambda': lam, 'DeltaPsi': val})
df = pd.DataFrame(results)
os.makedirs('output', exist_ok=True)
df.to_excel('output/demo_delta_psi_export.xlsx', index=False)
print("Dati esportati in output/demo_delta_psi_export.xlsx")

Dati esportati in output/demo_delta_psi_export.xlsx
